# Алгоритмы потоков и разрезов в графах


## Введение


**Определение:** Сеть (Network) — это ориентированный граф $G = (V, E)$ с неотрицательной пропускной способностью $c(u, v) \geq 0$ для каждого ребра $(u, v) \in E$, источником $s \in V$ и стоком $t \in V$.

**Определение:** Поток (Flow) в сети $G$ — это функция $f: V \times V \rightarrow \mathbb{R}$, удовлетворяющая следующим условиям:
1. **Ограничение пропускной способности:** Для всех $u, v \in V$, $0 \leq f(u, v) \leq c(u, v)$.
2. **Кососимметричность:** Для всех $u, v \in V$, $f(u, v) = -f(v, u)$.
3. **Сохранение потока:** Для всех $u \in V \setminus \{s, t\}$, $\sum_{v \in V} f(u, v) = 0$.

**Определение:** Величина потока (Flow Value) $|f|$ — это суммарный поток, выходящий из источника: $|f| = \sum_{v \in V} f(s, v)$.

**Определение:** Разрез (Cut) в сети $G$ — это разбиение множества вершин $V$ на два непересекающихся подмножества $S$ и $T = V \setminus S$ таких, что $s \in S$ и $t \in T$.

**Определение:** Пропускная способность разреза (Cut Capacity) $c(S, T)$ — это сумма пропускных способностей рёбер, идущих из $S$ в $T$: $c(S, T) = \sum_{u \in S} \sum_{v \in T} c(u, v)$.

**Определение:** Минимальный разрез (Minimum Cut) — это разрез с минимальной пропускной способностью.


## Теорема о максимальном потоке и минимальном разрезе


**Теорема:** Величина максимального потока в сети равна пропускной способности минимального разреза.

$\square$
Докажем, что для любого потока $f$ и любого разреза $(S, T)$ выполняется $|f| \leq c(S, T)$.

Рассмотрим произвольный поток $f$ и разрез $(S, T)$. По определению величины потока:
$|f| = \sum_{v \in V} f(s, v)$

По свойству сохранения потока, для каждой вершины $u \in S \setminus \{s\}$ выполняется $\sum_{v \in V} f(u, v) = 0$. Суммируя по всем вершинам из $S$:
$\sum_{u \in S} \sum_{v \in V} f(u, v) = |f|$

Разделим сумму на две части:
$\sum_{u \in S} \sum_{v \in S} f(u, v) + \sum_{u \in S} \sum_{v \in T} f(u, v) = |f|$

По свойству кососимметричности, $\sum_{u \in S} \sum_{v \in S} f(u, v) = 0$. Следовательно:
$\sum_{u \in S} \sum_{v \in T} f(u, v) = |f|$

По ограничению пропускной способности, $f(u, v) \leq c(u, v)$ для всех $(u, v) \in E$. Поэтому:
$|f| = \sum_{u \in S} \sum_{v \in T} f(u, v) \leq \sum_{u \in S} \sum_{v \in T} c(u, v) = c(S, T)$

Таким образом, величина любого потока не превышает пропускную способность любого разреза.

Теперь докажем, что существует поток $f$ и разрез $(S, T)$ такие, что $|f| = c(S, T)$.

Пусть $f$ — максимальный поток в сети. Определим множество $S$ как множество вершин, достижимых из $s$ в остаточной сети $G_f$, и $T = V \setminus S$. По построению, $s \in S$ и $t \in T$ (иначе существовал бы увеличивающий путь, и поток не был бы максимальным).

Для любого ребра $(u, v)$, где $u \in S$ и $v \in T$, должно выполняться $f(u, v) = c(u, v)$, иначе $(u, v)$ было бы ребром в остаточной сети, и $v$ было бы достижимо из $s$.

Для любого ребра $(u, v)$, где $u \in T$ и $v \in S$, должно выполняться $f(u, v) = 0$, иначе в остаточной сети было бы ребро $(v, u)$, и $u$ было бы достижимо из $s$.

Следовательно:
$|f| = \sum_{u \in S} \sum_{v \in T} f(u, v) - \sum_{u \in T} \sum_{v \in S} f(u, v) = \sum_{u \in S} \sum_{v \in T} c(u, v) - 0 = c(S, T)$

Таким образом, величина максимального потока равна пропускной способности минимального разреза.
$\blacksquare$


In [ ]:
from typing import Dict, List, Tuple, Set

def ford_fulkerson(graph: Dict[int, List[Tuple[int, int]]], source: int, sink: int) -> int:
    """
    Реализация алгоритма Форда-Фалкерсона для нахождения максимального потока.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки пар (соседняя вершина, пропускная способность)
        source: исток
        sink: сток

    Returns:
        величина максимального потока
    """
    # Создаем остаточную сеть
    residual_graph = {}
    for u in graph:
        residual_graph[u] = {}
        for v, capacity in graph[u]:
            residual_graph[u][v] = capacity
            if v not in residual_graph:
                residual_graph[v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = 0

    def bfs(residual_graph: Dict[int, Dict[int, int]], source: int, sink: int) -> List[int]:
        """Поиск увеличивающего пути с помощью BFS."""
        visited = {source: True}
        queue = [source]
        parent = {}

        while queue:
            u = queue.pop(0)
            for v in residual_graph[u]:
                if v not in visited and residual_graph[u][v] > 0:
                    queue.append(v)
                    visited[v] = True
                    parent[v] = u
                    if v == sink:
                        # Восстанавливаем путь
                        path = [sink]
                        while path[-1] != source:
                            path.append(parent[path[-1]])
                        path.reverse()
                        return path

        return []

    max_flow = 0

    # Пока существует увеличивающий путь
    while True:
        path = bfs(residual_graph, source, sink)
        if not path:
            break

        # Находим минимальную остаточную пропускную способность на пути
        min_capacity = float('infinity')
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            min_capacity = min(min_capacity, residual_graph[u][v])

        # Обновляем остаточную сеть
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            residual_graph[u][v] -= min_capacity
            residual_graph[v][u] += min_capacity

        max_flow += min_capacity

    return max_flow

# Пример использования
graph = {
    0: [(1, 16), (2, 13)],
    1: [(2, 10), (3, 12)],
    2: [(1, 4), (4, 14)],
    3: [(2, 9), (5, 20)],
    4: [(3, 7), (5, 4)],
    5: []
}

max_flow = ford_fulkerson(graph, 0, 5)
print(f"Максимальный поток: {max_flow}")  # Ожидаемый результат: 23


**Теорема:** Временная сложность алгоритма Форда-Фалкерсона составляет $O(|E| \cdot |f_{max}|)$, где $|E|$ — количество рёбер в графе, а $|f_{max}|$ — величина максимального потока.

$\square$
В алгоритме Форда-Фалкерсона каждый поиск увеличивающего пути требует $O(|E|)$ операций (при использовании BFS или DFS). Каждый найденный увеличивающий путь увеличивает поток как минимум на 1. Следовательно, количество итераций не превышает величину максимального потока $|f_{max}|$.

Итоговая сложность: $O(|E| \cdot |f_{max}|)$
$\blacksquare$


**Замечание:** Временная сложность алгоритма Форда-Фалкерсона зависит от величины максимального потока, что может быть неэффективно для графов с большими пропускными способностями. Существуют улучшенные версии алгоритма, такие как алгоритм Эдмондса-Карпа и алгоритм Диница, которые имеют полиномиальную сложность от размера графа.


## Алгоритм Эдмондса-Карпа


**Идея:** Алгоритм Эдмондса-Карпа является реализацией метода Форда-Фалкерсона, в которой для поиска увеличивающего пути используется алгоритм поиска в ширину (BFS). Это гарантирует, что на каждом шаге выбирается кратчайший увеличивающий путь.

**Теорема:** Алгоритм Эдмондса-Карпа находит максимальный поток за $O(|V| \cdot |E|^2)$ операций.

$\square$
Докажем, что алгоритм Эдмондса-Карпа выполняет не более $O(|V| \cdot |E|)$ итераций.

Определим расстояние $d(s, v)$ как минимальное количество рёбер в пути от $s$ до $v$ в остаточной сети. Поскольку BFS находит кратчайший путь по количеству рёбер, каждый увеличивающий путь, найденный алгоритмом, имеет длину $d(s, t)$.

Ключевое наблюдение: после обработки увеличивающего пути расстояние $d(s, t)$ не уменьшается.

Докажем это. Пусть после обработки некоторого увеличивающего пути расстояние $d(s, t)$ уменьшилось. Это означает, что появился новый путь с меньшим количеством рёбер. Новые рёбра в остаточной сети могут появиться только в обратном направлении к рёбрам обработанного пути. Но использование таких рёбер не может уменьшить расстояние, так как они ведут в направлении, противоположном пути от $s$ к $t$.

Теперь рассмотрим произвольное ребро $(u, v)$. После обработки увеличивающего пути, содержащего $(u, v)$, это ребро исчезает из остаточной сети, и может появиться снова только если путь будет содержать ребро $(v, u)$. Но в этом случае расстояние $d(s, v)$ должно быть не меньше $d(s, u) + 1$, иначе путь не был бы кратчайшим.

Таким образом, каждое ребро может быть критическим (т.е. иметь минимальную пропускную способность на пути) не более $O(|V|)$ раз, так как после каждого такого случая расстояние от $s$ до одного из его концов увеличивается как минимум на 2, а максимальное расстояние ограничено $|V|$.

Поскольку в графе $|E|$ рёбер, и каждое может быть критическим $O(|V|)$ раз, общее количество итераций не превышает $O(|V| \cdot |E|)$.

Каждая итерация включает поиск в ширину, который выполняется за $O(|E|)$ операций. Следовательно, общая временная сложность алгоритма Эдмондса-Карпа составляет $O(|V| \cdot |E|^2)$.
$\blacksquare$


In [ ]:
def edmonds_karp(graph: Dict[int, List[Tuple[int, int]]], source: int, sink: int) -> int:
    """
    Реализация алгоритма Эдмондса-Карпа для нахождения максимального потока.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки пар (соседняя вершина, пропускная способность)
        source: исток
        sink: сток

    Returns:
        величина максимального потока
    """
    # Создаем остаточную сеть
    residual_graph = {}
    for u in graph:
        residual_graph[u] = {}
        for v, capacity in graph[u]:
            residual_graph[u][v] = capacity
            if v not in residual_graph:
                residual_graph[v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = 0

    def bfs(residual_graph: Dict[int, Dict[int, int]], source: int, sink: int) -> Dict[int, int]:
        """Поиск кратчайшего увеличивающего пути с помощью BFS."""
        visited = {source: True}
        queue = [source]
        parent = {}

        while queue:
            u = queue.pop(0)
            for v in residual_graph[u]:
                if v not in visited and residual_graph[u][v] > 0:
                    queue.append(v)
                    visited[v] = True
                    parent[v] = u

        # Возвращаем словарь предков для восстановления пути
        return parent if sink in visited else {}

    max_flow = 0

    # Пока существует увеличивающий путь
    while True:
        parent = bfs(residual_graph, source, sink)
        if not parent:
            break

        # Восстанавливаем путь
        path = [sink]
        while path[-1] != source:
            path.append(parent[path[-1]])
        path.reverse()

        # Находим минимальную остаточную пропускную способность на пути
        min_capacity = float('infinity')
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            min_capacity = min(min_capacity, residual_graph[u][v])

        # Обновляем остаточную сеть
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            residual_graph[u][v] -= min_capacity
            residual_graph[v][u] += min_capacity

        max_flow += min_capacity

    return max_flow

# Пример использования
graph = {
    0: [(1, 16), (2, 13)],
    1: [(2, 10), (3, 12)],
    2: [(1, 4), (4, 14)],
    3: [(2, 9), (5, 20)],
    4: [(3, 7), (5, 4)],
    5: []
}

max_flow = edmonds_karp(graph, 0, 5)
print(f"Максимальный поток: {max_flow}")  # Ожидаемый результат: 23


## Алгоритм Диница


**Идея:** Алгоритм Диница улучшает алгоритм Эдмондса-Карпа, используя понятие слоистой сети и блокирующего потока. На каждой итерации алгоритм строит слоистую сеть, находит в ней блокирующий поток и обновляет остаточную сеть.

**Определение:** Слоистая сеть (Layered Network) — это подграф остаточной сети, содержащий только рёбра, которые ведут из вершины уровня $i$ в вершину уровня $i+1$, где уровень вершины — это её расстояние от источника.

**Определение:** Блокирующий поток (Blocking Flow) — это поток, в котором любой путь от источника к стоку содержит хотя бы одно насыщенное ребро (ребро, поток через которое равен его пропускной способности).

**Теорема:** Алгоритм Диница находит максимальный поток за $O(|V|^2 \cdot |E|)$ операций.

$\square$
Докажем, что алгоритм Диница выполняет не более $O(|V|)$ итераций внешнего цикла.

На каждой итерации алгоритм строит слоистую сеть и находит в ней блокирующий поток. После этого минимальное расстояние от источника до стока в остаточной сети увеличивается как минимум на 1. Это происходит потому, что все пути минимальной длины в текущей остаточной сети содержат хотя бы одно насыщенное ребро, которое исчезает из остаточной сети.

Поскольку расстояние от источника до стока не может превышать $|V|$, количество итераций внешнего цикла не превышает $O(|V|)$.

Построение слоистой сети требует $O(|E|)$ операций (с помощью BFS). Нахождение блокирующего потока в слоистой сети можно выполнить за $O(|V| \cdot |E|)$ операций, используя серию поисков в глубину.

Таким образом, общая временная сложность алгоритма Диница составляет $O(|V| \cdot |V| \cdot |E|) = O(|V|^2 \cdot |E|)$.
$\blacksquare$


In [ ]:
def dinic(graph: Dict[int, List[Tuple[int, int]]], source: int, sink: int) -> int:
    """
    Реализация алгоритма Диница для нахождения максимального потока.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки пар (соседняя вершина, пропускная способность)
        source: исток
        sink: сток

    Returns:
        величина максимального потока
    """
    # Создаем остаточную сеть
    residual_graph = {}
    for u in graph:
        residual_graph[u] = {}
        for v, capacity in graph[u]:
            residual_graph[u][v] = capacity
            if v not in residual_graph:
                residual_graph[v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = 0

    def bfs(residual_graph: Dict[int, Dict[int, int]], source: int, sink: int) -> Dict[int, int]:
        """Построение слоистой сети с помощью BFS."""
        level = {source: 0}
        queue = [source]
        i = 0

        while i < len(queue):
            u = queue[i]
            i += 1
            for v in residual_graph[u]:
                if v not in level and residual_graph[u][v] > 0:
                    level[v] = level[u] + 1
                    queue.append(v)

        return level if sink in level else {}

    def dfs(u: int, flow: int, level: Dict[int, int], ptr: Dict[int, Dict[int, int]]) -> int:
        """Поиск блокирующего потока с помощью DFS."""
        if u == sink:
            return flow

        for v in residual_graph[u]:
            if v not in ptr[u]:
                ptr[u][v] = True
                if v in level and level[v] == level[u] + 1 and residual_graph[u][v] > 0:
                    min_flow = min(flow, residual_graph[u][v])
                    pushed_flow = dfs(v, min_flow, level, ptr)

                    if pushed_flow > 0:
                        residual_graph[u][v] -= pushed_flow
                        residual_graph[v][u] += pushed_flow
                        return pushed_flow

        return 0

    max_flow = 0

    while True:
        level = bfs(residual_graph, source, sink)
        if not level:
            break

        ptr = {u: {} for u in residual_graph}

        while True:
            flow = dfs(source, float('infinity'), level, ptr)
            if flow == 0:
                break
            max_flow += flow

    return max_flow

# Пример использования
graph = {
    0: [(1, 16), (2, 13)],
    1: [(2, 10), (3, 12)],
    2: [(1, 4), (4, 14)],
    3: [(2, 9), (5, 20)],
    4: [(3, 7), (5, 4)],
    5: []
}

max_flow = dinic(graph, 0, 5)
print(f"Максимальный поток: {max_flow}")  # Ожидаемый результат: 23


## Приложения алгоритмов потоков и разрезов


### Задача о максимальном паросочетании в двудольном графе

**Определение:** Двудольный граф (Bipartite Graph) — это граф, вершины которого можно разбить на два непересекающихся множества $X$ и $Y$ так, что каждое ребро соединяет вершину из $X$ с вершиной из $Y$.

**Определение:** Паросочетание (Matching) в графе — это множество рёбер, не имеющих общих вершин.

**Определение:** Максимальное паросочетание (Maximum Matching) — это паросочетание с максимально возможным количеством рёбер.

**Теорема:** Задача о максимальном паросочетании в двудольном графе может быть сведена к задаче о максимальном потоке.

$\square$
Рассмотрим двудольный граф $G = (X \cup Y, E)$. Построим сеть $G' = (V', E')$ следующим образом:
1. Добавим источник $s$ и сток $t$: $V' = X \cup Y \cup \{s, t\}$.
2. Для каждой вершины $x \in X$ добавим ребро $(s, x)$ с пропускной способностью 1.
3. Для каждой вершины $y \in Y$ добавим ребро $(y, t)$ с пропускной способностью 1.
4. Для каждого ребра $(x, y) \in E$ добавим ребро $(x, y)$ с пропускной способностью 1.

Максимальный поток в сети $G'$ соответствует максимальному паросочетанию в графе $G$. Ребро $(x, y)$ включается в паросочетание тогда и только тогда, когда поток через это ребро равен 1.

Корректность этого сведения следует из того, что:
1. Поток через каждое ребро может быть либо 0, либо 1 (из-за целочисленности пропускных способностей и свойств алгоритмов поиска максимального потока).
2. Для каждой вершины $x \in X$ суммарный исходящий поток не превышает 1 (из-за ограничения на ребре $(s, x)$).
3. Для каждой вершины $y \in Y$ суммарный входящий поток не превышает 1 (из-за ограничения на ребре $(y, t)$).

Таким образом, рёбра с ненулевым потоком образуют паросочетание, и величина потока равна размеру этого паросочетания.
$\blacksquare$


In [ ]:
def maximum_bipartite_matching(graph: Dict[int, List[int]], X: Set[int], Y: Set[int]) -> List[Tuple[int, int]]:
    """
    Нахождение максимального паросочетания в двудольном графе с помощью алгоритма поиска максимального потока.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки смежных вершин
        X: множество вершин первой доли
        Y: множество вершин второй доли

    Returns:
        список пар (x, y), образующих максимальное паросочетание
    """
    # Создаем сеть для задачи о максимальном потоке
    source = -1
    sink = -2
    flow_graph = {}

    # Добавляем источник и рёбра от источника к вершинам X
    flow_graph[source] = [(x, 1) for x in X]

    # Добавляем рёбра из графа
    for u in graph:
        if u in X:
            flow_graph[u] = [(v, 1) for v in graph[u]]
        else:
            flow_graph[u] = []

    # Добавляем рёбра от вершин Y к стоку
    for y in Y:
        if y not in flow_graph:
            flow_graph[y] = []
        flow_graph[y].append((sink, 1))

    # Добавляем сток
    flow_graph[sink] = []

    # Находим максимальный поток
    residual_graph = {}
    for u in flow_graph:
        residual_graph[u] = {}
        for v, capacity in flow_graph[u]:
            residual_graph[u][v] = capacity
            if v not in residual_graph:
                residual_graph[v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = 0

    # Используем алгоритм Форда-Фалкерсона для нахождения максимального потока
    def bfs(residual_graph: Dict[int, Dict[int, int]], source: int, sink: int) -> Dict[int, int]:
        """Поиск увеличивающего пути с помощью BFS."""
        visited = {source: True}
        queue = [source]
        parent = {}

        while queue:
            u = queue.pop(0)
            for v in residual_graph[u]:
                if v not in visited and residual_graph[u][v] > 0:
                    queue.append(v)
                    visited[v] = True
                    parent[v] = u

        return parent if sink in visited else {}

    # Находим максимальный поток
    while True:
        parent = bfs(residual_graph, source, sink)
        if not parent:
            break

        # Восстанавливаем путь
        path = [sink]
        while path[-1] != source:
            path.append(parent[path[-1]])
        path.reverse()

        # Находим минимальную остаточную пропускную способность на пути
        min_capacity = float('infinity')
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            min_capacity = min(min_capacity, residual_graph[u][v])

        # Обновляем остаточную сеть
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            residual_graph[u][v] -= min_capacity
            residual_graph[v][u] += min_capacity

    # Извлекаем паросочетание из потока
    matching = []
    for x in X:
        for y in graph.get(x, []):
            if y in Y and residual_graph[x][y] == 0:  # Если поток через ребро равен 1
                matching.append((x, y))
                break  # Каждая вершина может быть сопоставлена только один раз

    return matching

# Пример использования
bipartite_graph = {
    0: [4, 5],
    1: [4, 6],
    2: [5],
    3: [6, 7]
}
X = {0, 1, 2, 3}  # Первая доля
Y = {4, 5, 6, 7}  # Вторая доля

matching = maximum_bipartite_matching(bipartite_graph, X, Y)
print(f"Максимальное паросочетание: {matching}")  # Ожидаемый результат: [(0, 5), (1, 4), (2, 5), (3, 6)]


### Задача о минимальном вершинном покрытии в двудольном графе

**Определение:** Вершинное покрытие (Vertex Cover) в графе — это множество вершин, таких что каждое ребро графа инцидентно хотя бы одной вершине из этого множества.

**Определение:** Минимальное вершинное покрытие (Minimum Vertex Cover) — это вершинное покрытие с минимально возможным количеством вершин.

**Теорема Кёнига:** В двудольном графе размер минимального вершинного покрытия равен размеру максимального паросочетания.

$\square$
Пусть $G = (X \cup Y, E)$ — двудольный граф, $M$ — максимальное паросочетание в $G$, и $C$ — минимальное вершинное покрытие в $G$.

Сначала докажем, что $|C| \geq |M|$. Каждое ребро из $M$ должно быть покрыто хотя бы одной вершиной из $C$. Поскольку рёбра в $M$ не имеют общих вершин, каждая вершина из $C$ может покрыть не более одного ребра из $M$. Следовательно, $|C| \geq |M|$.

Теперь докажем, что $|C| \leq |M|$, построив вершинное покрытие размера $|M|$.

Построим ориентированный граф $G'$ следующим образом:
1. Для каждого ребра $(x, y) \in M$, где $x \in X$ и $y \in Y$, ориентируем его от $y$ к $x$.
2. Для каждого ребра $(x, y) \in E \setminus M$, ориентируем его от $x$ к $y$.

Пусть $S$ — множество вершин, недостижимых из непокрытых вершин $Y$ в графе $G'$. Определим $C' = (X \setminus S) \cup (Y \cap S)$.

Докажем, что $C'$ является вершинным покрытием. Рассмотрим произвольное ребро $(x, y) \in E$:
- Если $(x, y) \in M$, то либо $x \notin S$ (и тогда $x \in C'$), либо $x \in S$, что означает, что $y \in S$ (из-за ориентации ребра), и тогда $y \in C'$.
- Если $(x, y) \notin M$, то либо $x \notin S$ (и тогда $x \in C'$), либо $x \in S$, что означает, что $y \notin S$ (иначе было бы противоречие с определением $S$), и тогда $y \in C'$.

Таким образом, $C'$ — вершинное покрытие.

Теперь докажем, что $|C'| = |M|$. Для каждого ребра $(x, y) \in M$, где $x \in X$ и $y \in Y$, либо $x \notin S$, либо $y \in S$, но не оба одновременно (из-за ориентации ребра). Следовательно, каждое ребро из $M$ вносит ровно одну вершину в $C'$, и $|C'| = |M|$.

Поскольку $C$ — минимальное вершинное покрытие, $|C| \leq |C'| = |M|$.

Таким образом, $|C| = |M|$.
$\blacksquare$


In [ ]:
def minimum_vertex_cover_bipartite(graph: Dict[int, List[int]], X: Set[int], Y: Set[int]) -> Set[int]:
    """
    Нахождение минимального вершинного покрытия в двудольном графе с помощью максимального паросочетания.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки смежных вершин
        X: множество вершин первой доли
        Y: множество вершин второй доли

    Returns:
        множество вершин, образующих минимальное вершинное покрытие
    """
    # Находим максимальное паросочетание
    matching = maximum_bipartite_matching(graph, X, Y)

    # Создаем словарь соответствия для паросочетания
    match_dict = {}
    for x, y in matching:
        match_dict[x] = y
        match_dict[y] = x

    # Строим ориентированный граф
    directed_graph = {v: [] for v in graph}
    for u in graph:
        for v in graph[u]:
            if u in X:
                if v == match_dict.get(u):
                    # Ребро из паросочетания: ориентируем от Y к X
                    directed_graph[v].append(u)
                else:
                    # Ребро не из паросочетания: ориентируем от X к Y
                    directed_graph[u].append(v)

    # Находим множество вершин, недостижимых из непокрытых вершин Y
    unmatched_Y = Y - {y for _, y in matching}
    visited = set()

    def dfs(node):
        if node in visited:
            return
        visited.add(node)
        for neighbor in directed_graph.get(node, []):
            dfs(neighbor)

    # Запускаем DFS из непокрытых вершин Y
    for y in unmatched_Y:
        dfs(y)

    # Формируем минимальное вершинное покрытие
    cover = (X - visited) | (Y & visited)

    return cover

# Пример использования
bipartite_graph = {
    0: [4, 5],
    1: [4, 6],
    2: [5],
    3: [6, 7]
}
X = {0, 1, 2, 3}  # Первая доля
Y = {4, 5, 6, 7}  # Вторая доля

cover = minimum_vertex_cover_bipartite(bipartite_graph, X, Y)
print(f"Минимальное вершинное покрытие: {cover}")  # Ожидаемый результат: {0, 1, 2, 6}


### Задача о максимальном потоке минимальной стоимости

**Определение:** Сеть с ценами (Network with Costs) — это сеть, в которой каждому ребру $(u, v)$ сопоставлена не только пропускная способность $c(u, v)$, но и цена $p(u, v)$, представляющая стоимость передачи единицы потока по этому ребру.

**Определение:** Стоимость потока (Flow Cost) — это сумма произведений потока по каждому ребру на цену этого ребра: $cost(f) = \sum_{(u, v) \in E} f(u, v) \cdot p(u, v)$.

**Определение:** Задача о максимальном потоке минимальной стоимости (Minimum Cost Maximum Flow Problem) заключается в нахождении потока максимальной величины с минимальной возможной стоимостью.

**Теорема:** Задача о максимальном потоке минимальной стоимости может быть решена с помощью последовательного нахождения путей с минимальной стоимостью в остаточной сети.

$\square$
Рассмотрим алгоритм, который последовательно увеличивает поток, выбирая на каждом шаге путь с минимальной стоимостью в остаточной сети.

Пусть $f$ — текущий поток, и $G_f$ — соответствующая остаточная сеть. Для каждого ребра $(u, v)$ в $G_f$ определим его стоимость следующим образом:
- Если $(u, v) \in E$, то стоимость равна $p(u, v)$.
- Если $(u, v) \notin E$, но $(v, u) \in E$, то стоимость равна $-p(v, u)$.

Докажем, что если $p$ — путь с минимальной стоимостью от $s$ до $t$ в $G_f$, то увеличение потока вдоль $p$ приводит к потоку с минимальной стоимостью среди всех потоков той же величины.

Предположим противное: существует поток $f'$ той же величины, что и $f$, но с меньшей стоимостью. Рассмотрим разность потоков $f' - f$. Эта разность представляет собой циркуляцию (поток с нулевой величиной) и может быть разложена на простые циклы и пути от $s$ до $t$ и от $t$ до $s$.

Поскольку $|f'| = |f|$, количество путей от $s$ до $t$ равно количеству путей от $t$ до $s$. Если бы в $G_f$ существовал цикл отрицательной стоимости, то мы могли бы увеличить поток вдоль этого цикла и получить поток той же величины, но с меньшей стоимостью, что противоречит выбору пути $p$.

Таким образом, если на каждом шаге мы выбираем путь с минимальной стоимостью, то в итоге получим максимальный поток с минимальной стоимостью.
$\blacksquare$


In [ ]:
import heapq

def min_cost_max_flow(graph: Dict[int, List[Tuple[int, int, int]]], source: int, sink: int) -> Tuple[int, int]:
    """
    Нахождение максимального потока минимальной стоимости.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки троек (соседняя вершина, пропускная способность, цена)
        source: исток
        sink: сток

    Returns:
        кортеж (величина максимального потока, минимальная стоимость)
    """
    # Создаем остаточную сеть
    residual_graph = {}
    cost = {}

    for u in graph:
        residual_graph[u] = {}
        cost[u] = {}
        for v, capacity, price in graph[u]:
            residual_graph[u][v] = capacity
            cost[u][v] = price
            if v not in residual_graph:
                residual_graph[v] = {}
                cost[v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = 0
                cost[v][u] = -price

    def dijkstra(residual_graph: Dict[int, Dict[int, int]], cost: Dict[int, Dict[int, int]], source: int, sink: int) -> Tuple[Dict[int, int], int]:
        """Поиск пути с минимальной стоимостью с помощью алгоритма Дейкстры."""
        distance = {u: float('infinity') for u in residual_graph}
        distance[source] = 0
        parent = {}
        visited = set()

        priority_queue = [(0, source)]

        while priority_queue:
            dist, u = heapq.heappop(priority_queue)

            if u in visited:
                continue

            visited.add(u)

            if u == sink:
                break

            for v in residual_graph[u]:
                if residual_graph[u][v] > 0 and v not in visited:
                    new_dist = distance[u] + cost[u][v]
                    if new_dist < distance[v]:
                        distance[v] = new_dist
                        parent[v] = u
                        heapq.heappush(priority_queue, (new_dist, v))

        return parent, distance[sink]

    max_flow = 0
    min_cost = 0

    while True:
        parent, path_cost = dijkstra(residual_graph, cost, source, sink)

        if not parent or sink not in parent:
            break

        # Восстанавливаем путь
        path = [sink]
        while path[-1] != source:
            path.append(parent[path[-1]])
        path.reverse()

        # Находим минимальную остаточную пропускную способность на пути
        min_capacity = float('infinity')
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            min_capacity = min(min_capacity, residual_graph[u][v])

        # Обновляем остаточную сеть и стоимость
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            residual_graph[u][v] -= min_capacity
            residual_graph[v][u] += min_capacity
            min_cost += min_capacity * cost[u][v]

        max_flow += min_capacity

    return max_flow, min_cost

# Пример использования
# Граф представлен как словарь смежности, где каждое ребро имеет (вершина, пропускная способность, цена)
cost_graph = {
    0: [(1, 10, 1), (2, 8, 2)],
    1: [(2, 5, 1), (3, 4, 3)],
    2: [(3, 7, 2)],
    3: []
}

max_flow, min_cost = min_cost_max_flow(cost_graph, 0, 3)
print(f"Максимальный поток: {max_flow}")
print(f"Минимальная стоимость: {min_cost}")


## Заключение


В данном разделе мы рассмотрели основные алгоритмы для решения задач о потоках и разрезах в графах:

1. **Алгоритм Форда-Фалкерсона**
   - Базовый алгоритм для нахождения максимального потока
   - Временная сложность: $O(|E| \cdot |f_{max}|)$
   - Основан на итеративном увеличении потока вдоль увеличивающих путей

2. **Алгоритм Эдмондса-Карпа**
   - Реализация метода Форда-Фалкерсона с использованием BFS для поиска кратчайшего увеличивающего пути
   - Временная сложность: $O(|V| \cdot |E|^2)$
   - Гарантирует полиномиальное время работы независимо от величины потока

3. **Алгоритм Диница**
   - Улучшение алгоритма Эдмондса-Карпа с использованием слоистых сетей и блокирующих потоков
   - Временная сложность: $O(|V|^2 \cdot |E|)$
   - Более эффективен на практике для многих типов графов

Мы также рассмотрели важную теоретическую основу — теорему о максимальном потоке и минимальном разрезе, которая устанавливает, что величина максимального потока в сети равна пропускной способности минимального разреза.

Кроме того, мы изучили несколько важных приложений алгоритмов потоков и разрезов:

1. **Задача о максимальном паросочетании в двудольном графе**
   - Может быть сведена к задаче о максимальном потоке
   - Позволяет эффективно находить максимальное количество пар без общих вершин

2. **Задача о минимальном вершинном покрытии в двудольном графе**
   - Согласно теореме Кёнига, размер минимального вершинного покрытия равен размеру максимального паросочетания
   - Может быть решена с помощью алгоритмов поиска максимального потока

3. **Задача о максимальном потоке минимальной стоимости**
   - Обобщение задачи о максимальном потоке с учетом стоимости передачи потока по рёбрам
   - Может быть решена с помощью последовательного нахождения путей с минимальной стоимостью

**Сравнение алгоритмов:**

| Алгоритм | Временная сложность | Особенности |
|----------|---------------------|-------------|
| Форд-Фалкерсон | $O(|E| \cdot |f_{max}|)$ | Простая реализация, но зависит от величины потока |
| Эдмондс-Карп | $O(|V| \cdot |E|^2)$ | Полиномиальное время, использует BFS |
| Диниц | $O(|V|^2 \cdot |E|)$ | Более эффективен, использует слоистые сети |

**Практическое применение:**
Алгоритмы потоков и разрезов имеют широкое применение в различных областях:
- Транспортные и логистические задачи
- Планирование ресурсов и расписаний
- Компьютерные сети и маршрутизация
- Биоинформатика (например, сегментация изображений)
- Задачи о назначениях и распределении ресурсов

Понимание теории потоков и разрезов, а также соответствующих алгоритмов, является важной частью арсенала любого специалиста в области алгоритмов и дискретной математики.
